In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5" 
import torch
torch.cuda.device_count()

1

In [2]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from glob import glob
# import wandb

import monai
from monai.losses import DiceCELoss, DiceFocalLoss, FocalLoss
from monai.inferers import sliding_window_inference
from monai import transforms

from monai.transforms import (
       AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    MapTransform,
    ScaleIntensityd,
    #AddChanneld,
    SpatialPadd,
    CenterSpatialCropd,
    EnsureChannelFirstd,
    ConcatItemsd,
    AdjustContrastd, 
    Rand3DElasticd,
    HistogramNormalized,
    NormalizeIntensityd,
    Invertd,
    SaveImage,

)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import SwinUNETR, UNETR, SegResNet
from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)
from monai import data


from monai.utils import first, set_determinism
from sklearn.model_selection import train_test_split
import json


import torch

In [3]:
set_determinism(seed=1024)

In [4]:
import os
import json
import random
from glob import glob

def generate_paths(patient_id):
    base_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data'
    return {
        'id': os.path.join(base_dir, patient_id),
        'ct': os.path.join(base_dir, patient_id, f"{patient_id}_ct.nii.gz"),
        'seg': os.path.join(base_dir, patient_id, f"{patient_id}_gt.nii.gz")
    }

# Assign each data entry to a random fold
all_data = []
num_folds = 5

for file_dir in sorted(glob('data/*')):
    patient_id = file_dir.split('/')[-1]
    # Check if the file belongs to the MDA center
    if patient_id.startswith("HMR-"):
        entry = generate_paths(patient_id)
        entry['fold'] = random.randint(1, num_folds) - 1
        all_data.append(entry)

# Compile data into a JSON structure
data_json = {"training": all_data}

# Save to JSON file
json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_ct_train_new.json"
with open(json_file_path, 'w') as f:
    json.dump(data_json, f, indent=4)

print(f"JSON file created at {json_file_path}")


JSON file created at /home/nada.saadi/CTPET/hecktor2022_cropped/HMR_ct_train_new.json


In [5]:
import json

json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_ct_train_new.json"

with open(json_file_path, 'r') as f:
    data = json.load(f)

first_5_elements = data["training"][:5]
print(first_5_elements)

[{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001/HMR-001_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001/HMR-001_gt.nii.gz', 'fold': 0}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004/HMR-004_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004/HMR-004_gt.nii.gz', 'fold': 3}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-005', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-005/HMR-005_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-005/HMR-005_gt.nii.gz', 'fold': 3}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-011', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-011/HMR-011_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-011/HMR-011_gt.nii.gz', 'fold': 2}, {'id': 

In [6]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
json_dir = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_ct_train_new.json"

In [7]:
def datafold_read(datalist, basedir, fold=0, key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

In [8]:
train_files, validation_files = datafold_read(datalist=json_dir, basedir=data_dir, fold=0)
len(train_files), len(validation_files)

(13, 4)

In [9]:
class ClipCT(MapTransform):
    """
    Convert labels to multi channels based on hecktor classes:
    label 1 is the tumor
    label 2 is the lymph node

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            if key == "ct":
                d[key] = torch.clip(d[key], min=-200, max=200)
            # elif key == "pt":
            #     d[key] = torch.clip(d[key], d[key].min(), 5)
        return d

class MulPTFM(MapTransform):
    """
    Mult PT and FM 

    """

    def __call__(self, data):
        d = dict(data)

        fm = d["ct"] > 0
        d["pt"] = d["pt"] * fm
        return d

class SelectClass(MapTransform):
    """
    Select the class for which you want to fine tune the model 

    """
    # def __init__(self, keys, cls=1):
    #     super(self).__init__(keys)
    #     self.cls = cls

    def __call__(self, data):
        d = dict(data)
        d["seg"][d["seg"] == 1] = 0
        # d["seg"][d["seg"] == 2] = 1
        
        return d

In [10]:
import json

json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_ct_train_new.json"

with open(json_file_path, 'r') as f:
    data = json.load(f)

first_5_elements = data["training"][:5]
print(first_5_elements)

[{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001/HMR-001_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001/HMR-001_gt.nii.gz', 'fold': 0}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004/HMR-004_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004/HMR-004_gt.nii.gz', 'fold': 3}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-005', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-005/HMR-005_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-005/HMR-005_gt.nii.gz', 'fold': 3}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-011', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-011/HMR-011_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-011/HMR-011_gt.nii.gz', 'fold': 2}, {'id': 

In [11]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
json_dir = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_ct_train_new.json"

In [12]:
num_samples = 4

train_transforms = Compose(
    [
        LoadImaged(keys=["ct", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["ct",  "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["ct",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["ct"]),
        ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
        #NormalizeIntensityd(keys=["ctpt"], channel_wise=True),
        RandCropByPosNegLabeld(
            keys=["ct", "seg"],
            label_key="seg",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=num_samples,
            image_key="ct",
            image_threshold=0,
        ),
        RandFlipd(
            keys=["ct", "seg"],
            spatial_axis=[0],
            prob=0.20,
        ),
        RandFlipd(
            keys=["ct", "seg"],
            spatial_axis=[1],
            prob=0.20,
        ),
        RandFlipd(
            keys=["ct", "seg"],
            spatial_axis=[2],
            prob=0.20,
        ),
        RandRotate90d(
            keys=["ct", "seg"],
            prob=0.20,
            max_k=3,
        ),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["ct", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["ct", "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["ct",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["ct"]),
        ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
    ]
)

In [13]:
train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)

train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=8,
    pin_memory=torch.cuda.is_available(),
    )

val_ds = monai.data.Dataset(data=validation_files, transform=val_transforms)

val_loader = DataLoader(
    val_ds, 
    batch_size=2, 
    num_workers=8, 
    shuffle= False)

In [14]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNETR(
    in_channels=1,
    out_channels=3,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072, 
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device) 


/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.unetr UNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)


In [15]:
torch.backends.cudnn.benchmark = True
def poly_lr(epoch, max_epochs, initial_lr, exponent=0.9):
    return initial_lr * (1 - epoch / max_epochs)**exponent

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [16]:
model_dir = '/home/nada.saadi/CTPET/hecktor2022_cropped/7thctchannel'

def validation(epoch_iterator_val):
    model.eval()
    with torch.no_grad():
        for step, batch in enumerate(epoch_iterator_val):
            val_inputs, val_labels = (batch["ct"].cuda(), batch["seg"].cuda())
            val_outputs = sliding_window_inference(val_inputs, (96, 96, 96), 4, model)
            val_labels_list = decollate_batch(val_labels)
            val_labels_convert = [
                post_label(val_label_tensor) for val_label_tensor in val_labels_list
            ]
            val_outputs_list = decollate_batch(val_outputs)
            val_output_convert = [
                post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list
            ]
            dice_metric(y_pred=val_output_convert, y=val_labels_convert)
            dice_metric_batch(y_pred=val_output_convert, y=val_labels_convert)
            epoch_iterator_val.set_description(
                "Validate (%d / %d Steps)" % (global_step, 10.0)
            )
        mean_dice_val = dice_metric.aggregate().item()
        metric_batch_val = dice_metric_batch.aggregate()

        metric_tumor = metric_batch_val[0].item()
        metric_lymph = metric_batch_val[1].item()

        dice_metric.reset()
        dice_metric_batch.reset()
    return mean_dice_val, metric_tumor, metric_lymph


def train(global_step, train_loader, dice_val_best, global_step_best):
    model.train()
    epoch_loss = 0
    step = 0
    epoch_iterator = tqdm(
        train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True
    )
    for step, batch in enumerate(epoch_iterator):
        step += 1
        x, y = (batch["ct"].cuda(), batch["seg"].cuda())
        logit_map = model(x)
        loss = loss_function(logit_map, y)
        loss.backward()
        epoch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        epoch_iterator.set_description(
            "Training (%d / %d Steps) (loss=%2.5f)"
            % (global_step, max_iterations, loss)
        )
        if (
           step = 0
    epoch_iterator = tqdm(
        train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True
    )
    for step, batch in enumerate(epoch_iterator):
        step += 1
        x, y = (batch["ct"].cuda(), batch["seg"].cuda())
        logit_map = model(x)
        loss = loss_function(logit_map, y)
        loss.backward()
        epoch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        epoch_iterator.set_description(
            "Training (%d / %d Steps) (loss=%2.5f)"
            % (global_step, max_iterations, loss)
        )
        if (
                    step = 0
    epoch_iterator = tqdm(
        train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True
    )
    for step, batch in enumerate(epoch_iterator):
        step += 1
        x, y = (batch["ct"].cuda(), batch["seg"].cuda())
        logit_map = model(x)
        loss = loss_function(logit_map, y)
        loss.backward()
        epoch_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        epoch_iterator.set_description(
            "Training (%d / %d Steps) (loss=%2.5f)"
            % (global_step, max_iterations, loss)
        )
        if (
post_label = AsDiscrete(to_onehot=3)
post_pred = AsDiscrete(argmax=True, to_onehot=3)

dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
dice_metric_batch = DiceMetric(include_background=False, reduction="mean_batch")

epoch = 0
max_num_epochs = 530

global_step = 0
dice_val_best = 0.0
global_step_best = 0
epoch_loss_values = []
metric_values = []
metric_values_tumor = []
metric_values_lymph = []
while global_step < max_iterations:
    global_step, dice_val_best, global_step_best = train(
        global_step, train_loader, dice_val_best, global_step_best
    )
    # wandb.log({'learning_rate': optimizer.param_groups[0]['lr']})
    # wandb.log({'Best Dice': dice_val_best})
    epoch += 1
    #optimizer.param_groups[0]['lr'] = poly_lr(epoch, max_num_epochs, 0.005676 , 0.9)
# model.load_state_dict(torch.load(os.path.join(model_dir, "best_metric_luck_UNETr_prompt.pth")))

Training (100 / 18000 Steps) (loss=1.20381):  43%|████▎     | 3/7 [00:09<00:14,  3.58s/it]

Model Was Saved ! Current Best Avg. Dice: 0.00031642516842111945 Current Avg. Dice: 0.00031642516842111945 Current Avg. tumor Dice: 0.0006328503368422389 Current Avg. lymph Dice: 0.0


Training (200 / 18000 Steps) (loss=1.11445):  71%|███████▏  | 5/7 [00:09<00:04,  2.45s/it]

Model Was Saved ! Current Best Avg. Dice: 0.0008080410188995302 Current Avg. Dice: 0.0008080410188995302 Current Avg. tumor Dice: 0.0016160820377990603 Current Avg. lymph Dice: 0.0


Training (300 / 18000 Steps) (loss=1.06042): 100%|██████████| 7/7 [00:10<00:00,  1.54s/it]


...

Training (17900 / 18000 Steps) (loss=0.32640):  29%|██▊       | 2/7 [00:08<00:22,  4.44s/it]

Model Was Not Saved ! Current Best Avg. Dice: 0.395572304725647 Current Avg. Dice: 0.34053996205329895 Current Avg. tumor Dice: 0.33818238973617554 Current Avg. lymph Dice: 0.3428975045681


Training (18000 / 18000 Steps) (loss=0.35390):  57%|█████▋    | 4/7 [00:08<00:07,  2.55s/it]

Model Was Not Saved ! Current Best Avg. Dice: 0.395572304725647 Current Avg. Dice: 0.3749883770942688 Current Avg. tumor Dice: 0.3674938976764679 Current Avg. lymph Dice: 0.3824828267097473


Training (18003 / 18000 Steps) (loss=0.24630): 100%|██████████| 7/7 [00:09<00:00,  1.43s/it]
